In [ ]:
# %% 

from notebook_config import DATASETS_DIR, EXPERIMENTAL_RESULTS_DIR, FILES_DIR, MODEL_CONFIGS
import pandas as pd
import time
from tqdm import tqdm
from port.entity_extractor import MultiEntityExtractor
from sklearn.model_selection import train_test_split

[2025-08-04 23:52:15] INFO in config: PyTorch version 2.7.1+cu128 available.


In [ ]:
# %%

df = pd.read_csv(DATASETS_DIR / 'full_data_clean.csv').fillna('')

In [ ]:
# %%

df.head()

,Unnamed: 0,urls,text,persons,organizations,themes,locations
0,381,https://arabic.cnn.com/travel/article/2022/08/...,اليابان فتحت أبوابها أمام المسافرين.. ولكن ما ...,,,"IMMIGRATION,999;WB_2670_JOBS,999;WB_2769_JOBS_...",Japan;Taiwan;Hong Kong;Singapore;Australia;Kyo...
1,664,https://www.cnn.com/2022/08/08/media/axios-cox...,"Axios, the buzzy digital media venture founded...",Roy Schwartz;Jim Vandehei;Alex Taylor;Mike Allen,Schwartz;Schwartz;Schwartz;Schwartz;New York T...,"TAX_FNCACT_CHIEF,1346;TAX_FNCACT_CHIEF,2085;UN...",Washington;New York
2,257,https://www.cnn.com/2022/08/03/energy/joe-bide...,Candidate Joe Biden promised in 2019 to make S...,Matt Smith;Mizuho Yawger;Jamal Khashoggi;Jim S...,Rapidan Energy Group;White House;White House;W...,"GENERAL_GOVERNMENT,2297;EPU_POLICY_GOVERNMENT,...",White House;The White House;Americans;Washingt...
3,338,https://www.cnn.com/2022/08/02/media/alex-jone...,The parents of a child who was murdered during...,Roy Lubit;Scarlett Lewis;Avi Moshenberg;Alex J...,Cnn;Cnn;Cnn,"EXTREMISM,197;EPU_CATS_MIGRATION_FEAR_FEAR,204...",Sandy Hook;Connecticut;Texas
4,319,https://www.cnn.com/style/article/india-coutur...,What India Couture Week reveals about this yea...,Amit Aggarwal;Amit Aggarwal;Amit Aggarwal;Amit...,Cnn;Fashion Design Council Of India;United States,"TAX_ETHNICITY_ASIAN,806;TAX_FNCACT_ACTOR,3616;...",India;New Delhi;Paris;United States


In [ ]:
# %%

def create_multi_entity_extractor(config):
    """
    Factory function to create a MultiEntityExtractor from a configuration dictionary.
    
    Args:
        config (dict): Configuration dictionary with extractor classes and parameters
        
    Returns:
        MultiEntityExtractor: Configured multi-entity extractor
    """
    extractor = MultiEntityExtractor()
    
    # Add extractors for each entity type
    for entity_type in ["persons", "organizations", "locations"]:
        if entity_type in config:
            extractor_class = config[entity_type]["extractor"]
            params = config[entity_type]["params"]
            extractor.add_extractor(entity_type, extractor_class(**params))
    
    return extractor

In [ ]:
# %%

# Create predictions folder
predictions_dir = FILES_DIR / 'predictions'
predictions_dir.mkdir(exist_ok=True)

# Test all models and collect results
results = []
runs = 5
for run in range(runs):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    train_df = train_df.sample(frac=0.3)
    for config in tqdm(MODEL_CONFIGS, desc="Testing models"):
        model_name = config['name']
        model_info = config['extra_info']
        
        start_time = time.time()
        
        try:
            # Create extractor using factory function
            extractor = create_multi_entity_extractor(config)
            
            # Fit and predict
            fit_start = time.time()
            extractor.fit(train_df['text'], train_df[['persons', 'organizations', 'locations']])
            fit_time = time.time() - fit_start
            
            predict_start = time.time()
            predictions = extractor.predict(test_df['text'])
            predict_time = time.time() - predict_start
            
            total_time = time.time() - start_time
            
            # Save predictions to file as DataFrame with true values
            model_size = model_info.get('model', 'N/A')
            safe_model_size = model_size.replace('/', '_').replace('-', '_')
            predictions_file = predictions_dir / f"{model_name}_{safe_model_size}_predictions.csv"
            
            # Create DataFrame with predictions
            predictions_df = pd.DataFrame(predictions)
            
            # Add true values from test set with TRUE_ prefix
            true_values = test_df[['persons', 'organizations', 'locations']].copy()
            true_values.columns = ['TRUE_persons', 'TRUE_organizations', 'TRUE_locations']
            
            # Combine predictions and true values
            combined_df = pd.concat([predictions_df.reset_index(), true_values.reset_index()], axis=1, ignore_index=True)
            combined_df.to_csv(predictions_file, index=False)
            
            # Store results with all metadata
            results.append({
                'run': run,
                'model_name': model_name,
                **model_info,
                'status': 'success',
                'fit_time': fit_time,
                'predict_time': predict_time,
                'total_time': total_time,
                'error': None,
                'stats': extractor.stats,
                'predictions_file': str(predictions_file)
            })
            
        except Exception as e:
            total_time = time.time() - start_time
            error_msg = str(e)
            
            results.append({
                'run': run,
                'model_name': model_name,
                **model_info,
                'status': 'error',
                'fit_time': 0,
                'predict_time': 0,
                'total_time': total_time,
                'error': error_msg,
                'stats': {},
                'predictions_file': None
            })
        
        results_df = pd.json_normalize(results)
        results_df.to_csv(EXPERIMENTAL_RESULTS_DIR / 'model_evaluation_results_multi_pass.csv', index=False)

Testing models:   0%|          | 0/7 [00:00<?, ?it/s]

[2025-08-04 23:52:18] INFO in pretrained: CUDA is available. Using GPU: NVIDIA GeForce RTX 2070 with Max-Q Design
[2025-08-04 23:52:18] INFO in pretrained: CUDA is available. Using GPU: NVIDIA GeForce RTX 2070 with Max-Q Design
[2025-08-04 23:52:19] INFO in pretrained: CUDA is available. Using GPU: NVIDIA GeForce RTX 2070 with Max-Q Design
[2025-08-04 23:52:19] INFO in entity_extractor: Fitting MultiEntityExtractor with 83 samples
[2025-08-04 23:52:19] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-04 23:52:19] INFO in preprocessing: Converted to list of size 83 in 0.0010008811950683594 seconds
[2025-08-04 23:52:19] INFO in pretrained: Loading model from: c:\Users\User\Workspace\work_practice\interview\toptal\take-home-assignment\files\pretrained\distilbert_ner_finetuned


Device set to use cuda:0


[2025-08-04 23:52:20] INFO in pretrained: Model loaded successfully
[2025-08-04 23:52:20] INFO in entity_extractor: Fitting PretrainedBertEntityExtractorPure with 83 samples
[2025-08-04 23:52:23] INFO in entity_extractor: Evaluating PretrainedBertEntityExtractorPure with 83 samples
[2025-08-04 23:52:23] INFO in entity_extractor: Evaluation results: {'toptal': {'multi_word': {'precision': 0.743243, 'recall': 0.348837, 'jaccard': 0.311321, 'f1': 0.47482, 'accuracy': 0.084337}}, 'personal': {'micro': {'accuracy': 0.08433734939759036, 'precision': 0.7297297297297297, 'recall': 0.34249471458773784, 'jaccard': 0.30393996247654786, 'f1': 0.46618705035971225}, 'macro': {'accuracy': 0.08433734939759036, 'precision': 0.5844807802639128, 'recall': 0.37030785231156377, 'jaccard': 0.3219647910041062, 'f1': 0.41479946754201663}}}
[2025-08-04 23:52:23] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-04 23:52:23] INFO in preprocessing: Converted to list of size 83 in 0.0009

Device set to use cuda:0


...

Testing models:  71%|███████▏  | 5/7 [03:45<01:22, 41.26s/it]

[2025-08-05 00:19:52] INFO in spacy: Loading spacy model 'en_core_web_sm'
[2025-08-05 00:19:54] INFO in spacy: Loaded spacy model 'en_core_web_sm'
[2025-08-05 00:19:54] INFO in spacy: Loading spacy model 'en_core_web_sm'
[2025-08-05 00:19:55] INFO in spacy: Loaded spacy model 'en_core_web_sm'
[2025-08-05 00:19:55] INFO in spacy: Loading spacy model 'en_core_web_sm'
[2025-08-05 00:19:57] INFO in spacy: Loaded spacy model 'en_core_web_sm'
[2025-08-05 00:19:57] INFO in entity_extractor: Fitting MultiEntityExtractor with 83 samples
[2025-08-05 00:19:57] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-05 00:19:57] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-05 00:19:57] INFO in entity_extractor: Fitting SpacyEntityExtractor with 83 samples
[2025-08-05 00:20:14] INFO in entity_extractor: Evaluating SpacyEntityExtractor with 83 samples
[2025-08-05 00:20:14] INFO in entity_extractor: Evaluation results: {'toptal': {'multi_word': {'pre

Testing models:  86%|████████▌ | 6/7 [05:23<01:00, 60.59s/it]

[2025-08-05 00:21:30] INFO in spacy: Loading spacy model 'en_core_web_md'
[2025-08-05 00:21:33] INFO in spacy: Loaded spacy model 'en_core_web_md'
[2025-08-05 00:21:33] INFO in spacy: Loading spacy model 'en_core_web_md'
[2025-08-05 00:21:35] INFO in spacy: Loaded spacy model 'en_core_web_md'
[2025-08-05 00:21:35] INFO in spacy: Loading spacy model 'en_core_web_md'
[2025-08-05 00:21:37] INFO in spacy: Loaded spacy model 'en_core_web_md'
[2025-08-05 00:21:37] INFO in entity_extractor: Fitting MultiEntityExtractor with 83 samples
[2025-08-05 00:21:37] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-05 00:21:37] INFO in preprocessing: Converted to list of size 83 in 0.0 seconds
[2025-08-05 00:21:37] INFO in entity_extractor: Fitting SpacyEntityExtractor with 83 samples
[2025-08-05 00:21:55] INFO in entity_extractor: Evaluating SpacyEntityExtractor with 83 samples
[2025-08-05 00:21:55] INFO in entity_extractor: Evaluation results: {'toptal': {'multi_word': {'pre

Testing models: 100%|██████████| 7/7 [07:00<00:00, 60.04s/it]


In [ ]:
# %%

# Save results to CSV